<a href="https://colab.research.google.com/github/hyeonji0401/CUDA_practice/blob/main/CUDA_memory_hierarchy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 8.1 컴퓨터 시스템의 메모리 계층

보조기억장치 -> 메인 메모리 -> 캐시 -> 레지스터

-> 으로 갈수록 적은 용량, 고성능, 고가임

- 보조기억장치(auxilary memory)
 - 흔히 디스크로 부르며 데이터를 보관하기 위해 사용함
 - 컴퓨터가 데이터를 사용하기 위해서는 보조기억장치에 있는 데이터를 메인 메모리로 가지고 와야함
- 메인 메모리(main memory)
 - 연산 장치가 접근할 수 있는 가장 먼 저장장치로 연산을 위한 데이터를 보관하는 공간
 - 흔히 메모리라고 부름

**=> 보조메모리와 메인 메모리를 구분한 이유**

- 최소한의 비용으로 최대한의 효과를 얻기 위해
 - 저렴하지만 용량이 큰 보조기억장치에 모든 데이터를 보관해두고 현재 작업에 필요한 데이터만 속도가 빠른 메인 메모리에 가져와 사용하는 방법으로 용량과 속도를 둘다 잡는 전략

 - 레지스터 & 캐시
  - CPU 내부에 있는 메모리롤 메인 메모리보다 월등히 빠르지만 그만큼 비싸고 매우 작은 저장공간
  - 레지스터 : 연산 장치가 현재 수행 중인 연산을 위해 사용하는 임시 저장 공간
  - 캐시 : 자주 사용하는 데이터를 가져다 놓는 공간
  - 우리가 직접 제어할 수 없고 CPU와 같은 하드웨어에 의해 관리됨


=> GPU 메모리도 계층을 나눠놓은 이유가 유사하지만 CPU의 캐시와 달리 GPU는 특정 캐시 메모리(공유 메모리)를 직접 제어할 수 있는 특성을 가짐

# 8.2 CUDA 메모리 계층

1. 스레드 수준 메모리
- 각 스레드 내부에서 사용되므로 다른 스레드에서 접근할 수 없는 메모리 공간

가. 레지스터
- CUDA 코어 연산을 위한 데이터를 담아두고 사용하는 공간
- 스레드가 연산을 위해 데이터를 저장하는 공간
- 커널 내부에서 선언된 지역 변수를 위해 사용됨
- GPU 메모리 중 가장 빠른 메모리로 일반적으로 GPU cycle보다 작은 시간 소요
- SM내부에 있어 인-코어(in-core) 메모리라고 부름
- 속도는 빠르지만 가장 크기가 작은 메모리임(블록 또는 SM하나당 8k-64k개의 32비트)

 ex) 하나의 스레드 블록 당 최대 스레드 1024개를 사용하는 경우

    - 한 스레드는 8 ~ 64개의 레지스터만 사용 가능 (8 ~ 64개의 정수형 변수,,)
- 활성 블록 수를 늘리게 되면 스레드 당 사용가능한 레지스터 수는 더 줄어들음

=> 활성 블록의 수를 2로 만들면 스레드당 할당 가능한 레지스터 수는 절반으로 줄어듦

- 실제 지역 변수의 수가 사용 가능한 레지스터 수보다 많은 경우, 어떤 지역 변수를 레지스터에 담을지 여부는 컴파일러에 의해 결정됨
- 레지스터 담지 못하는 지역 변수는 지역 메모리에 할당됨

2. 지역 메모리
- SM 밖에 있는 오프-칩(off-chip) 메모리임
- 접근 속도는 400 ~ 600 GPU cycle정도로 레지스터보다 느리지만 사용 가능한 메모리 공간이 큼
- GPU 디바이스 메모리(DRAM: GPU사양에서 말하는 '메모리'를 의미함) 공간 일부가 지역 메모리로 사용됨
- 전역 메모리 또한 디바이스 메모리 영역을 사용하지만 지역 메모리는 각 스레드가 자신만의 공간을 갖는다는 점에서 전역메모리와 구분됨
- 레지스터를 사용하기에는 큰 구조체나 배열 등이나 일반 변수이지만 레지스터 공간을 할당 받지 못할 경우 지역 메모리 공간을 사용함
(스레드당 512KB 제한이 있지만 스레드 하나가 지역 변수를 사용하기에는 충분함)

